In [ ]:
using ModelingToolkit
using MethodOfLines
using DifferentialEquations
using DomainSets
using ProgressMeter
using Plots

In [ ]:
@parameters t x
@variables η(..) u(..)
Dt = Differential(t);
Dx = Differential(x);

g = 9.81
H = 3
omega = 2*pi*10

x_l = 0
x_r = 2.0
tmax = 1.0
N = 100
dx = (x_r - x_l)/N
dt = dx/sqrt(g*H)
println("No. timesteps is")
println(tmax/dt)

IC(x) = x
F(x, t) = 3*sin(omega * t)
eqs = [Dt(η(t, x)) + H * Dx(u(t, x)) ~ 0,
       Dt(u(t, x)) + g * Dx(η(t, x)) ~ 0]

bcs = [η(0.0, x) ~ IC(x),
       u(0.0, x) ~ 0.0,
       u(t, x_r) ~ 0.0,
       η(t, 0.0) ~ F(x, t)]

domains = [t in Interval(0.0, tmax),
           x in Interval(x_l, x_r)];
       
@named pdesys = PDESystem(eqs, bcs, domains, [t,x], [η(t,x), u(t,x)]);

discretization = MOLFiniteDifference([x=>dx], t, grid_align=MethodOfLines.StaggeredGrid(), edge_aligned_var=u(t,x));
prob = discretize(pdesys, discretization, analytic=nothing);
sol = solve(prob, ImplicitEuler(), dt=dt);

In [ ]:
println(fieldnames(typeof(sol)))
println(discretization.grid_align)
println(sol[x])

In [ ]:
grid = x_l:dx:x_r
println(grid)
u_matrix = sol[η(t, x)]
end_t = length(u_matrix[:,1])

anim = @animate for t in range(1, end_t)
    arr = u_matrix[t, :]
    plot(grid, arr,
         ylim = (-5, 5),
         xlabel = "x",
         ylabel = "u(x,t)",
         title = "u(x, t),  t = $(round(t, digits=2))",
         legend = false)
end

# Save as GIF
gif(anim, "SWE_1D.gif", fps=30)